Importing Required Tools

In [44]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import os

LLM model key

In [45]:
with open("secrets.json") as f:
    secrets = json.load(f)
Groq_Token = secrets["groq"]
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

Few Shot Learning function

In [46]:
def fsl(acc):
    acc1=pd.read_csv("./Combined/Train/LAYING/Subject_1.csv")
    acc2=pd.read_csv("./Combined/Train/SITTING/Subject_1.csv")
    acc3=pd.read_csv("./Combined/Train/STANDING/Subject_1.csv")
    acc4=pd.read_csv("./Combined/Train/WALKING/Subject_1.csv")
    acc5=pd.read_csv("./Combined/Train/WALKING_DOWNSTAIRS/Subject_1.csv")
    acc6=pd.read_csv("./Combined/Train/WALKING_UPSTAIRS/Subject_1.csv")
    query = f"""
    * You are a Human Activity Recognition model trained to classify human activities based on accelerometer data.
* Your task is to analyze the acceleration data provided in the x, y, and z directions and classify the activity being performed. The possible activities outcomes are 'Laying', 'Sitting', 'Standing', 'Walking', 'Walking_Downstairs', and 'Walking_Upstairs'.
* The accelerometer data represents movement patterns along three axes:
  - The x-axis typically captures forward and backward movement.
  - The y-axis captures lateral (side-to-side) movement.
  - The z-axis captures vertical movement, such as jumping or transitioning between sitting and standing.
* You need to provide only the answer in one word
    1. Acceleration: {acc1}
    Output = Laying
    2. Acceleration: {acc2}
    Output = Sitting
    3. Acceleration: {acc3}
    Output = Standing
    4. Acceleration: {acc4}
    Output = Walking
    5. Acceleration: {acc5}
    Output = Walking_downstream
    6. Acceleration: {acc6}
    Output = Walking_upstream

    Acceleration: {acc}
    Output = ?
""" 


    model_name = "llama3-70b"
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)

    return answer.content
acc= "./Combined/Test/WALKING_DOWNSTAIRS/Subject_10.csv"
print(fsl(acc))

Walking_Downstairs


In [47]:
lis = []
dire = "./Task_4/data/"

for filename in os.listdir(dire):
    if filename.endswith('.csv'):
        # Read the CSV file
        df = pd.read_csv(os.path.join(dire, filename))
        df = df.iloc[::8]
        starting = int(df.shape[0]/3)
        df = df.iloc[starting:starting+500]
        df = df.iloc[:, 1:4]
        df = df.reset_index(drop=True)

        print(df)
        df.to_csv(f"./exp/{filename}",index=False)


        gFx     gFy     gFz
0    0.9712  0.0605 -0.1238
1    0.9751  0.0624 -0.1262
2    0.9751  0.0605 -0.1204
3    0.9703  0.0610 -0.1238
4    0.9727  0.0615 -0.1214
..      ...     ...     ...
495  0.4147 -0.2586 -0.8260
496  0.4426 -0.2229 -0.8567
497  0.4489 -0.1960 -0.8523
498  0.4552 -0.1687 -0.7614
499  0.5564 -0.1433 -0.7385

[500 rows x 3 columns]
        gFx     gFy     gFz
0   -0.9300 -0.3915  0.0130
1   -0.9226 -0.3905  0.0213
2   -0.9236 -0.3910  0.0125
3   -0.9270 -0.3846  0.0179
4   -0.9246 -0.3817  0.0242
..      ...     ...     ...
495 -0.1624 -0.2967  0.9511
496 -0.1633 -0.2937  0.9536
497 -0.1604 -0.2903  0.9526
498 -0.1619 -0.2918  0.9536
499 -0.1609 -0.2928  0.9536

[500 rows x 3 columns]
        gFx     gFy     gFz
0   -0.9856  0.0034  0.2426
1   -0.9925  0.0034  0.2534
2   -0.9959  0.0029  0.2529
3   -0.9939  0.0019  0.2441
4   -0.9905  0.0019  0.2441
..      ...     ...     ...
495 -0.9920  0.0014  0.2373
496 -0.9959  0.0024  0.2358
497 -0.9920  0.0038  0.2368


In [ ]:
folder = "./exp/"
files = os.listdir(folder)

actual_labels = []
zsl_predictions = []
fsl_predictions = []

# all_files = os.listdir(os.path.join(folder, fol))
csv_files = [file for file in files if file.endswith('.csv')]
# print(csv_files)

for file_name in csv_files:
    file_path = os.path.join(folder, file_name)
    
    data = pd.read_csv(file_path)
    
    # actual_label = fol
    
    fsl_prediction = fsl(data)
    
    # actual_labels.append(actual_label)
    fsl_predictions.append(fsl_prediction.strip().upper())

In [53]:
actual_labels= ["LAYING"]*4+["SITTING"]*4+["STANDING"]*4+["WALKING"]*4+["WALKING_DOWNSTAIRES"]*4+["WALKING_UPSTAIRES"]*4
fsl_accuracy = accuracy_score(actual_labels, fsl_predictions)
fsl_accuracy

0.3333333333333333

Data performed well with the accuracy of 33%. Accuracy is low may be due to high covarience between walking, walking_upstires, walking_downstaires and due to error in reading data.

In [58]:
def fsl_1(acc):
    acc1=pd.read_csv("./exp/laying_1.csv")
    acc2=pd.read_csv("./exp/sit_1.csv")
    acc3=pd.read_csv("./exp/stand_1.csv")
    acc4=pd.read_csv("./exp/walk_1.csv")
    acc5=pd.read_csv("./exp/walk_down_1.csv")
    acc6=pd.read_csv("./exp/walk_up_1.csv")
    query = f"""
    * You are a Human Activity Recognition model trained to classify human activities based on accelerometer data.
* Your task is to analyze the acceleration data provided in the x, y, and z directions and classify the activity being performed. The possible activities outcomes are 'Laying', 'Sitting', 'Standing', 'Walking', 'Walking_Downstairs', and 'Walking_Upstairs'.
* The accelerometer data represents movement patterns along three axes:
  - The x-axis typically captures forward and backward movement.
  - The y-axis captures lateral (side-to-side) movement.
  - The z-axis captures vertical movement, such as jumping or transitioning between sitting and standing.
* You need to provide only the answer in one word
    1. Acceleration: {acc1}
    Output = Laying
    2. Acceleration: {acc2}
    Output = Sitting
    3. Acceleration: {acc3}
    Output = Standing
    4. Acceleration: {acc4}
    Output = Walking
    5. Acceleration: {acc5}
    Output = Walking_downstream
    6. Acceleration: {acc6}
    Output = Walking_upstream

    Acceleration: {acc}
    Output = ?
""" 
    model_name = "llama3-70b"
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)

    return answer.content

In [59]:
folder = "./exp/"
files = os.listdir(folder)

actual_labels_1 = []
fsl_predictions_1 = []

# all_files = os.listdir(os.path.join(folder, fol))
csv_files = [file for file in files if (file.endswith('.csv') and not file.endswith("1.csv"))]

# print(csv_files)

for file_name in csv_files:
    file_path = os.path.join(folder, file_name)
    
    data = pd.read_csv(file_path)
    
    # actual_label = fol
    
    fsl_prediction_1 = fsl_1(data)
    
    # actual_labels.append(actual_label)
    fsl_predictions_1.append(fsl_prediction_1.strip().upper())

In [68]:
actual_labels_1= ["LAYING"]*3+["SITTING"]*3+["STANDING"]*3+["WALKING"]*3+["WALKING_DOWNSTAIRES"]*3+["WALKING_UPSTAIRES"]*3
fsl_accuracy_1 = accuracy_score(actual_labels_1, fsl_predictions_1)
fsl_accuracy_1

0.2777777777777778

Here, the accuracy is 28%. This accuracy is due to error in taking readings and filteration of data, and also LLM model is predicting based on its data which may differ from the data we practically collected.